In [ ]:
!pip install numpy==1.26.4 --quiet

In [ ]:
!pip install -q torch sentence-transformers faiss-cpu fasttext tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.4 MB

In [ ]:
import os, re, json, math, time, shutil, sqlite3, glob, hashlib, datetime
from typing import List, Tuple, Iterable, Dict, Optional
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

SHARDS_DIR   = "/content/drive/MyDrive/Nour/bluesky/full_index"
DB_DRIVE     = "/content/drive/MyDrive/Nour/bluesky/seen/seen.db"
DB_LOCAL     = "/content/seen.db"
USE_LOCAL_DB = True
STATE_PATH   = "/content/drive/MyDrive/Nour/bluesky/seen/resume_state.json"
EXPORT_DIR   = "/content/drive/MyDrive/Nour/bluesky/exports"
os.makedirs(os.path.dirname(DB_DRIVE), exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)


Mounted at /content/drive


In [ ]:

DEFAULT_TAU = 0.66
TAU_STEP    = 0.02
MAX_TAU_BUMPS = 3

PER_SHARD_CANDIDATE_BUDGET = 10_000
GLOBAL_CANDIDATE_BUDGET    = 50_000
SEARCH_MODE = "range"

PREFLIGHT_ENABLE    = True
PREFLIGHT_TOPK      = 200

BATCH_INSERT_SIZE = 5000
TOPK_PER_SHARD = 20000
TAU_OVERRIDES = {}

def clean_query(text: str) -> str:
    if not text:
        return ""
    text = text.lower().strip()
    text = re.sub(r'https?://\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
def str_to_int64(s: str) -> int:
    h = hashlib.blake2b(s.encode('utf-8'), digest_size=8).digest()
    return int(np.frombuffer(h, dtype=np.uint64)[0].view(np.int64))


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model  = SentenceTransformer('all-MiniLM-L12-v2', device=device)
EMB_DIM = model.get_sentence_embedding_dimension()

def embed_query(q: str) -> np.ndarray:
    vec = model.encode([q], batch_size=1, convert_to_numpy=True, normalize_embeddings=True, show_progress_bar=False)
    return vec.astype('float32')


def list_shard_files(shards_dir: str) -> List[str]:

    files = sorted(glob.glob(os.path.join(shards_dir, "all_posts_index_shard_*.faiss")))
    return files

def load_index(path: str):
    try:
        ix = faiss.read_index(path)
        if not isinstance(ix, faiss.IndexIDMap2):
            ix = faiss.IndexIDMap2(ix)
        return ix
    except Exception as e:
        bak = path + ".bak"
        if os.path.exists(bak):
            try:
                ix = faiss.read_index(bak)
                if not isinstance(ix, faiss.IndexIDMap2):
                    ix = faiss.IndexIDMap2(ix)
                print(f"[WARN] Loaded backup for {os.path.basename(path)}")
                return ix
            except Exception as e2:
                print(f"[ERROR] Failed to read both {path} and {bak}: {e2}")
                return None
        print(f"[ERROR] Failed to read {path}: {e}")
        return None


def search_shard_topk(ix, qvec: np.ndarray, topk: int) -> Tuple[np.ndarray, np.ndarray]:
    D, I = ix.search(qvec, topk)
    scores, ids = D[0], I[0]
    m = ids != -1
    return scores[m], ids[m].astype(np.int64)

def search_shard_range(ix, qvec: np.ndarray, tau: float) -> Tuple[np.ndarray, np.ndarray]:
    lims, D, I = ix.range_search(qvec, tau)
    n = lims[1] - lims[0]
    if n <= 0:
        return np.empty((0,), dtype='float32'), np.empty((0,), dtype=np.int64)
    scores = np.array(D[lims[0]:lims[1]], dtype='float32')
    ids    = np.array(I[lims[0]:lims[1]], dtype=np.int64)
    return scores, ids


def is_relevant_stub(text: Optional[str], keyword_raw: str, keyword_clean: str, score: float) -> bool:
    return True

def lookup_text_for_ids(post_ids: Iterable[int]) -> Dict[int, Optional[str]]:
    return {int(pid): None for pid in post_ids}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_db_path() -> str:
    if USE_LOCAL_DB:
        return DB_LOCAL
    return DB_DRIVE

def ensure_schema(conn: sqlite3.Connection):
    cur = conn.cursor()
    cur.execute("PRAGMA journal_mode=WAL;")
    cur.execute("PRAGMA synchronous=NORMAL;")
    cur.execute("PRAGMA temp_store=MEMORY;")
    cur.execute("PRAGMA busy_timeout=5000;")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS keywords (
      id       INTEGER PRIMARY KEY,
      raw      TEXT NOT NULL,
      cleaned  TEXT NOT NULL UNIQUE,
      tau      REAL NOT NULL
    );
    """)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS accepted (
      post_id  INTEGER PRIMARY KEY
    );
    """)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS accepted_map (
      keyword_id  INTEGER NOT NULL,
      post_id     INTEGER NOT NULL,
      score       REAL,
      shard_id    INTEGER,
      accepted_at TEXT,
      PRIMARY KEY (keyword_id, post_id),
      FOREIGN KEY (keyword_id) REFERENCES keywords(id),
      FOREIGN KEY (post_id) REFERENCES accepted(post_id)
    );
    """)
    cur.execute("CREATE INDEX IF NOT EXISTS idx_accepted_map_post ON accepted_map(post_id);")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_keywords_cleaned   ON keywords(cleaned);")
    conn.commit()

def open_conn(path: str) -> sqlite3.Connection:
    conn = sqlite3.connect(path, timeout=30)
    ensure_schema(conn)
    return conn

def upsert_keyword(conn: sqlite3.Connection, raw: str, cleaned: str, tau: float) -> int:
    cur = conn.cursor()
    cur.execute("INSERT OR IGNORE INTO keywords(raw, cleaned, tau) VALUES(?,?,?)", (raw, cleaned, float(tau)))
    cur.execute("UPDATE keywords SET tau=? WHERE cleaned=? AND tau<>?", (float(tau), cleaned, float(tau)))
    conn.commit()
    cur.execute("SELECT id FROM keywords WHERE cleaned=?", (cleaned,))
    row = cur.fetchone()
    return int(row[0])

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def insert_accepted(conn: sqlite3.Connection, ids: List[int]):
    if not ids: return
    cur = conn.cursor()
    with conn:
        cur.executemany("INSERT OR IGNORE INTO accepted(post_id) VALUES (?)",
                        [(int(pid),) for pid in ids])

def insert_accepted_map(conn: sqlite3.Connection, keyword_id: int, rows: List[Tuple[int, float, int, str]]):
    if not rows: return
    cur = conn.cursor()
    with conn:
        cur.executemany(
            "INSERT OR IGNORE INTO accepted_map(keyword_id, post_id, score, shard_id, accepted_at) VALUES (?,?,?,?,?)",
            [(keyword_id, int(pid), float(sc), int(sid), ts) for (pid, sc, sid, ts) in rows]
        )


In [ ]:
def snapshot_db_to_drive(local_path: str, drive_path: str):
    # Atomic copy: write to tmp then replace
    tmp = drive_path + ".tmp"
    shutil.copy2(local_path, tmp)
    os.replace(tmp, drive_path)
    print(f"Snapshot saved to Drive: {drive_path}")

def prepare_db():
    if USE_LOCAL_DB:
        if not os.path.exists(DB_LOCAL):
            print("[DB] Creating local DB")
            conn = open_conn(DB_LOCAL)
            conn.close()
        if os.path.exists(DB_DRIVE) and os.path.getsize(DB_DRIVE) > os.path.getsize(DB_LOCAL):
            shutil.copy2(DB_DRIVE, DB_LOCAL)
            print("[DB] Pulled latest snapshot from Drive")
    else:
        if not os.path.exists(DB_DRIVE):
            print("[DB] Creating Drive DB")
            conn = open_conn(DB_DRIVE)
            conn.close()

prepare_db()


[DB] Creating local DB


In [ ]:
def process_keyword(raw_keyword: str,
                    tau: float = DEFAULT_TAU,
                    per_shard_budget: int = PER_SHARD_CANDIDATE_BUDGET,
                    global_budget: int = GLOBAL_CANDIDATE_BUDGET,
                    search_mode: str = SEARCH_MODE,
                    topk_per_shard: int = TOPK_PER_SHARD,   # kept for optional preflight use
                    use_sidecar_text: bool = False):
    """
    Uses FAISS range_search by default to keep all matches with cosine ≥ tau,
    with adaptive tau bumps if a shard floods. Results are sorted by score (desc)
    before acceptance. Writes dedupbed results into SQLite (accepted & accepted_map).
    """
    cleaned = clean_query(raw_keyword)
    if not cleaned:
        return {"keyword": raw_keyword, "cleaned": cleaned, "accepted": 0, "candidates": 0, "note": "empty after clean"}

    qvec = embed_query(cleaned)

    # DB
    db_path = get_db_path()
    conn = open_conn(db_path)
    keyword_id = upsert_keyword(conn, raw=raw_keyword, cleaned=cleaned, tau=tau)

    # Shards
    shard_paths = list_shard_files(SHARDS_DIR)
    if not shard_paths:
        return {"keyword": raw_keyword, "cleaned": cleaned, "accepted": 0, "candidates": 0, "note": "no shards found"}

    total_candidates = 0
    total_inserted   = 0
    p10_scores       = []
    med_scores       = []
    now_str          = datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"

    for shard_idx, spath in enumerate(tqdm(shard_paths, desc=f"Keyword: {raw_keyword}")):
        ix = load_index(spath)
        if ix is None:
            print(f"[WARN] Skipping shard: {os.path.basename(spath)}")
            continue

        tau_shard = float(tau)


        if PREFLIGHT_ENABLE:
            pf_scores, pf_ids = search_shard_topk(ix, qvec, min(PREFLIGHT_TOPK, ix.ntotal or 0))
            if len(pf_scores) > 0:
                kth_score = float(pf_scores[min(len(pf_scores)-1, PREFLIGHT_TOPK-1)])
                if kth_score >= tau_shard:
                    tau_shard = round(tau_shard + TAU_STEP, 4)

        bumps = 0
        while True:
            if search_mode == "range":
                scores, ids = search_shard_range(ix, qvec, tau_shard)
            else:
                scores, ids = search_shard_topk(ix, qvec, topk_per_shard)
                m = scores >= tau_shard
                scores, ids = scores[m], ids[m]

            if len(ids) > per_shard_budget and bumps < MAX_TAU_BUMPS:
                tau_shard = round(tau_shard + TAU_STEP, 4)
                bumps += 1
                continue
            break

        if len(scores) > 1:
            order = np.argsort(-scores)
            scores = scores[order]
            ids    = ids[order]

        if total_candidates + len(ids) > global_budget:
            keep = max(0, global_budget - total_candidates)
            scores, ids = scores[:keep], ids[:keep]

        total_candidates += len(ids)

        texts = lookup_text_for_ids(ids) if use_sidecar_text else {int(pid): None for pid in ids}

        accepted_ids  = []
        accepted_rows = []
        for sc, pid in zip(scores, ids):
            txt = texts.get(int(pid))
            if is_relevant_stub(txt, raw_keyword, cleaned, float(sc)):
                accepted_ids.append(int(pid))
                accepted_rows.append((int(pid), float(sc), shard_idx+1, now_str))

        for chunk_ids in chunks(accepted_ids, BATCH_INSERT_SIZE):
            insert_accepted(conn, chunk_ids)
        for chunk_rows in chunks(accepted_rows, BATCH_INSERT_SIZE):
            insert_accepted_map(conn, keyword_id, chunk_rows)

        total_inserted += len(accepted_ids)

        if len(scores) > 0:
            med = float(np.median(scores))
            p10 = float(np.percentile(scores, 10)) if len(scores) >= 10 else float(scores[-1])
            med_scores.append(med)
            p10_scores.append(p10)

        del ix

        if total_candidates >= global_budget:
            break

    conn.close()

    return {
        "keyword": raw_keyword,
        "cleaned": cleaned,
        "tau": tau,
        "candidates": total_candidates,
        "accepted": total_inserted,
        "median_score~": (sum(med_scores)/len(med_scores)) if med_scores else None,
        "p10_score~": (sum(p10_scores)/len(p10_scores)) if p10_scores else None
    }


In [ ]:
KEYWORDS = [
    'vaccine hoax',
    'vaccine agenda',
    'plandemic vaccine',
    'scamdemic jab',
    'deadly vaccine',
    'dangerous vaccine',
    'experimental vaccine',
    'poison vaccine',
    'DNA altering vaccine',
    'microchip vaccine',
    'vaccine microchip',
    '5G vaccine',
    'vaccine shedding',
    'shedding truth',
    'unsafe vaccine',
    'bioweapon vaccine"',
    'Bill Gates vaccine"',
    'forced vaccination"',
    'vaccine mandate"',
    'vaccine passport"',
    'population control vaccine"',
    'pharma greed"',
    'pharma cartel"',
    'Big Pharma agenda"',
    'Nuremberg vaccine"',
    'Nuremberg 2.0"',
    'crimes against humanity vaccine"',
    'adverse event vaccine"',
    'vaccine injury coverup"',
    'coverup vaccine deaths"',
    'not tested vaccine"',
    'rushed vaccine"',
    'spike protein damage"',
    'died suddenly vaccine"',
    'depopulation agenda vaccine"',
    'government control vaccine"',
    'toxic jab"',
    'magnetic vaccine"',
    'magnetized by vaccine"',
    'contaminated vaccine"',
    'vaccine infertility"',
    'vaccine sterilization"',
    'child sacrifice vaccine"',
    'protect the children from vaccine"',
    'vaccine genocide',
    'plandemic',
    'scamdemic',
    'shedding',
    'Died suddenly',
    'No Vaccine',
    'Vaccine Injury',
    'Nuremberg2',
    'Vaccine Agenda',
    'Vaccine Mandate',
    'Medical Freedom',
    'Health Freedom',
    'Big Pharma',
    'Do Your Own Research',
    'My Body My Choice',
    'Stop The Shot',
    'Experimental Vaccine',
    'Vaccine Coverup',
]


In [ ]:
def load_state() -> dict:
    if os.path.exists(STATE_PATH):
        try:
            with open(STATE_PATH, "r") as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_state(state: dict):
    tmp = STATE_PATH + ".tmp"
    with open(tmp, "w") as f:
        json.dump(state, f, indent=2)
    os.replace(tmp, STATE_PATH)



def run_all_keywords():
    if USE_LOCAL_DB:
        dbp = DB_LOCAL
    else:
        dbp = DB_DRIVE
    _ = open_conn(dbp).close()

    st = load_state()
    done_cleaned = set(st.get("done_cleaned", []))

    summaries = []
    for kw in KEYWORDS:
        cleaned = clean_query(kw)
        if cleaned in done_cleaned:
            print(f"[RESUME] Skipping already done keyword: {kw} -> {cleaned}")
            continue

        tau = TAU_OVERRIDES.get(kw, DEFAULT_TAU)
        summary = process_keyword(kw, tau=float(tau))
        summaries.append(summary)

        done_cleaned.add(cleaned)
        st["done_cleaned"] = sorted(list(done_cleaned))
        save_state(st)

        if USE_LOCAL_DB:
            snapshot_db_to_drive(DB_LOCAL, DB_DRIVE)

    print("\n=== SUMMARY ===")
    for s in summaries:
        print(s)

run_all_keywords()


Keyword: vaccine hoax:  79%|███████▉  | 15/19 [07:19<01:50, 27.70s/it]

[ERROR] Failed to read both /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0016.faiss and /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0016.faiss.bak: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0016.faiss.bak: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss


Keyword: vaccine hoax: 100%|██████████| 19/19 [08:40<00:00, 27.41s/it]


Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db


Keyword: vaccine agenda:  79%|███████▉  | 15/19 [02:54<00:47, 11.91s/it]

...

Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db
[RESUME] Skipping already done keyword: Experimental Vaccine -> experimental vaccine


Keyword: Vaccine Coverup:  79%|███████▉  | 15/19 [03:35<00:59, 14.79s/it]

[ERROR] Failed to read both /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0016.faiss and /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0016.faiss.bak: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0016.faiss.bak: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss


Keyword: Vaccine Coverup: 100%|██████████| 19/19 [04:07<00:00, 13.02s/it]

Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db

=== SUMMARY ===
{'keyword': 'vaccine hoax', 'cleaned': 'vaccine hoax', 'tau': 0.66, 'candidates': 156, 'accepted': 156, 'median_score~': 0.6751841174231635, 'p10_score~': 0.6637867685821321}
{'keyword': 'vaccine agenda', 'cleaned': 'vaccine agenda', 'tau': 0.66, 'candidates': 212, 'accepted': 212, 'median_score~': 0.6733113772728864, 'p10_score~': 0.6644823687918047}
{'keyword': 'plandemic vaccine', 'cleaned': 'plandemic vaccine', 'tau': 0.66, 'candidates': 82, 'accepted': 82, 'median_score~': 0.6676497845088735, 'p10_score~': 0.6635277071419885}
{'keyword': 'scamdemic jab', 'cleaned': 'scamdemic jab', 'tau': 0.66, 'candidates': 0, 'accepted': 0, 'median_score~': None, 'p10_score~': None}
{'keyword': 'deadly vaccine', 'cleaned': 'deadly vaccine', 'tau': 0.66, 'candidates': 251, 'accepted': 251, 'median_score~': 0.685505125257704, 'p10_score~': 0.6656439277860853}
{'keyword': 'dangerous vaccine', 'cleaned': 'dang

In [ ]:
import csv

def export_final_ids_csv(out_path: str):
    conn = open_conn(get_db_path())
    cur = conn.cursor()
    cur.execute("SELECT post_id FROM accepted;")
    rows = cur.fetchall()
    conn.close()

    with open(out_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["post_id"])
        for (pid,) in rows:
            w.writerow([int(pid)])
    print(f"Exported {len(rows)} unique post IDs to {out_path}")

def export_per_keyword_csv(out_path: str):
    conn = open_conn(get_db_path())
    cur = conn.cursor()
    cur.execute("""
    SELECT k.cleaned as keyword, m.post_id, m.score, m.shard_id, m.accepted_at
    FROM accepted_map m
    JOIN keywords k ON k.id = m.keyword_id
    ORDER BY keyword, score DESC, post_id;
    """)
    rows = cur.fetchall()
    conn.close()

    with open(out_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["keyword", "post_id", "score", "shard_id", "accepted_at"])
        for r in rows:
            w.writerow(r)
    print(f"Exported {len(rows)} (keyword, post_id, …) rows to {out_path}")

export_final_ids_csv(os.path.join(EXPORT_DIR, "dedup_post_ids.csv"))
export_per_keyword_csv(os.path.join(EXPORT_DIR, "per_keyword_matches.csv"))


Exported 3453 unique post IDs to /content/drive/MyDrive/Nour/bluesky/exports/dedup_post_ids.csv
Exported 5264 (keyword, post_id, …) rows to /content/drive/MyDrive/Nour/bluesky/exports/per_keyword_matches.csv


In [ ]:

def run_all_keywords_on_shard(shard_id_str="0016"):
    """
    Re-process ONLY the given shard for ALL keywords.
    Uses the same process_keyword() pipeline, but temporarily restricts
    list_shard_files() to a single file (copied locally for stability).
    """
    shard_name = f"all_posts_index_shard_{int(shard_id_str):04d}.faiss"
    shard_drive = os.path.join(SHARDS_DIR, shard_name)
    shard_bak   = shard_drive + ".bak"
    if not os.path.exists(shard_drive) and not os.path.exists(shard_bak):
        raise FileNotFoundError(f"Shard {shard_name} (.faiss or .faiss.bak) not found in {SHARDS_DIR}")

    local_dir  = "/content/shard_cache_catchup"
    os.makedirs(local_dir, exist_ok=True)
    local_path = os.path.join(local_dir, shard_name)

    src_used = None
    try:
        if os.path.exists(shard_drive):
            shutil.copy2(shard_drive, local_path)
            src_used = shard_drive
        else:
            shutil.copy2(shard_bak, local_path)
            src_used = shard_bak
        print(f"[INFO] Copied {os.path.basename(src_used)} → {local_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to copy shard locally: {e}")

    global list_shard_files
    _orig_list_shard_files = list_shard_files

    def _only_this_shard(_ignored_dir):
        return [local_path]

    list_shard_files = _only_this_shard

    summaries = []
    try:
        for kw in KEYWORDS:
            tau = TAU_OVERRIDES.get(kw, DEFAULT_TAU)
            s = process_keyword(kw, tau=float(tau))
            summaries.append(s)

            if 'USE_LOCAL_DB' in globals() and USE_LOCAL_DB:
                if 'snapshot_db_to_drive' in globals():
                    snapshot_db_to_drive(DB_LOCAL, DB_DRIVE)
    finally:
        list_shard_files = _orig_list_shard_files

    print("\n=== CATCH-UP SUMMARY (shard", shard_id_str, ") ===")
    for s in summaries:
        print(s)
    print("\nDone. Re-run your export cell to refresh CSVs.")


In [ ]:
run_all_keywords_on_shard("0016")

[INFO] Copied all_posts_index_shard_0016.faiss → /content/shard_cache_catchup/all_posts_index_shard_0016.faiss


Keyword: vaccine hoax: 100%|██████████| 1/1 [00:00<00:00, 1980.31it/s]


[ERROR] Failed to read /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss
Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db


Keyword: vaccine agenda: 100%|██████████| 1/1 [00:00<00:00, 2576.35it/s]


[ERROR] Failed to read /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss
Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db


...

[ERROR] Failed to read /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss
Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db



Keyword: Experimental Vaccine: 100%|██████████| 1/1 [00:00<00:00, 2985.27it/s]


[ERROR] Failed to read /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss
Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db


Keyword: Vaccine Coverup: 100%|██████████| 1/1 [00:00<00:00, 2101.35it/s]

[ERROR] Failed to read /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: Error in faiss::Index* faiss::read_index(IOReader*, int) at /project/third-party/faiss/faiss/impl/index_read.cpp:680: Error: 'ret == (1)' failed: read error in /content/shard_cache_catchup/all_posts_index_shard_0016.faiss: 0 != 1 (Resource temporarily unavailable)
[WARN] Skipping shard: all_posts_index_shard_0016.faiss
Snapshot saved to Drive: /content/drive/MyDrive/Nour/bluesky/seen/seen.db

=== CATCH-UP SUMMARY (shard 0016 ) ===
{'keyword': 'vaccine hoax', 'cleaned': 'vaccine hoax', 'tau': 0.66, 'candidates': 0, 'accepted': 0, 'median_score~': None, 'p10_score~': None}
{'keyword': 'vaccine agenda', 'cleaned': 'vaccine agenda', 'tau': 0.66, 'candidates': 0, 'accepted': 0, 'median_score~': None, 'p10_score~': None}
{'keyword': 'plandemic vaccine', 'cleaned': 'plandemic vaccine', 'tau': 0.66, 'candidates': 0, 'accepted': 0, 'median_score~': None, 'p10_score~': None}
{'keyword': 'scamdemic jab', 'cleane

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, json, csv, re, hashlib
import numpy as np
from tqdm import tqdm

DATA_DIR   = '/content/drive/MyDrive/Nour/users_posts_json'
DEDUP_CSV  = '/content/drive/MyDrive/Nour/bluesky/exports/per_keyword_matches.csv'
EXPORT_DIR = '/content/drive/MyDrive/Nour/bluesky/exports'
os.makedirs(EXPORT_DIR, exist_ok=True)

OUT_JSONL  = os.path.join(EXPORT_DIR, 'filtered_posts_full.jsonl')
MISSING_TXT= os.path.join(EXPORT_DIR, 'missing_post_ids_from_csv.txt')

def uri_to_int64(uri: str) -> np.int64:
    h = hashlib.blake2b(uri.encode('utf-8'), digest_size=8).digest()
    return np.frombuffer(h, dtype=np.uint64)[0].view(np.int64)

def load_ids_from_csv(path: str) -> set[int]:
    ids = set()
    with open(path, 'r', encoding='utf-8') as f:
        sniffer = csv.Sniffer()
        sample  = f.read(4096)
        f.seek(0)
        has_header = sniffer.has_header(sample)
        dialect    = sniffer.sniff(sample)
        reader     = csv.reader(f, dialect)
        header = None
        if has_header:
            header = next(reader, None)
            # find a likely id column
            id_idx = 0
            if header:
                # try common names
                for i, name in enumerate(header):
                    n = (name or '').strip().lower()
                    if n in ('post_id'):
                        id_idx = i
                        break
        else:
            id_idx = 0

        for row in reader:
            if not row:
                continue
            try:
                ids.add(int(row[id_idx]))
            except Exception:
                try:
                    ids.add(int(str(row[id_idx]).strip()))
                except Exception:
                    pass
    return ids

target_ids = load_ids_from_csv(DEDUP_CSV)
print(f"Loaded {len(target_ids)} unique target IDs from CSV.")

user_files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.json')])
found_ids  = set()
written    = 0

with open(OUT_JSONL, 'w', encoding='utf-8') as fout:
    for fname in tqdm(user_files, desc="Scanning JSON"):
        path = os.path.join(DATA_DIR, fname)
        try:
            with open(path, 'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"[WARN] Skipping unreadable file {fname}: {e}")
            continue

        for entry in data:
            post = entry.get("post")
            if not isinstance(post, dict):
                continue
            uri = post.get("uri")
            if not uri:
                continue

            pid = int(uri_to_int64(uri))
            if pid not in target_ids or pid in found_ids:
                continue

            rec = {
                "post_id": pid,
                "post": post
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
            found_ids.add(pid)
            written += 1

missing = sorted(list(target_ids - found_ids))
with open(MISSING_TXT, 'w', encoding='utf-8') as f:
    for pid in missing:
        f.write(str(pid) + "\n")

print(f"\nDone. Wrote {written} posts to {OUT_JSONL}")
print(f"Missing {len(missing)} IDs from CSV → {MISSING_TXT}")


Loaded 3453 unique target IDs from CSV.


Scanning JSON: 100%|██████████| 5785/5785 [3:21:53<00:00,  2.09s/it]


Done. Wrote 3453 posts to /content/drive/MyDrive/Nour/bluesky/exports/filtered_posts_full.jsonl
Missing 0 IDs from CSV → /content/drive/MyDrive/Nour/bluesky/exports/missing_post_ids_from_csv.txt
